In [137]:
import pandas as pd
import numpy as np
import nltk, datetime, re
from nltk.corpus import stopwords

In [125]:
#Run notebook in conda env lexnlp
import lexnlp.extract.en.definitions
import lexnlp.extract.en.amounts
import lexnlp.extract.en.regulations

/Users/katherinewood/anaconda3/envs/lexnlp/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
pd.set_option('display.max_columns', 999)

In [272]:
recalls = pd.read_csv('recalls.csv', 
                      dtype={'RecallID':'Int64', 'RecallNumber': str, 'RecallDate': str,
                             'Description': str, 'URL': str, 'Title': str, 'ConsumerContact': str,
                             'LastPublishDate': str, 'Images': 'object', 'SoldAtLabel': str,
                             'Distributors_CompanyID': 'Int64', 'DistributorsSName': str, 
                             'Hazards_HazardType': str, 'Hazards_HazardTypeID': str, 'Hazards_Name': str,
                             'Importers_CompanyID': 'Int64', 'Importers_Name': str, 'Inconjunctions_URL': str,
                             'Injuries_Name': str, 'ManufacturerCountries_Country': str, 
                             'Manufacturers_CompanyID': 'Int64', 'Manufacturers_Name': str, 'ProductUPCs_UPC': str,
                             'Products_CategoryID': 'Int64', 'Products_Description': str, 'Products_Model': str,
                             'Products_Name': str, 'Products_NumberOfUnits': str, 'Products_Type': str,
                             'Remedies_Name': str, 'RemedyOptions_Option': str, 'Retailers_CompanyID': str,
                             'Retailers_Name': str})

In [273]:
#This works reasonably well; seems to have low false-positives, but will also miss
#codes embedded in unusual sentences like "the UPC code for this product is XXXXX"
def identify_probable_upc(text):
    try:
        tk = nltk.word_tokenize(text)
    except:
        tk = nltk.word_tokenize("")
    tk = [token for token in tk if token not in ["’", ',', '.', '”', '“', 's']]
    pos = nltk.pos_tag(tk)
    grammar = "UPC: {<NN|NNP>+<:>*<VBD|VBZ>*<CD>+<CC>?<CD>?}"
    parser = nltk.RegexpParser(grammar)
    candidate_UPCs = [tree.leaves() for tree in parser.parse(pos).subtrees() if tree.label() in ['UPC']]
    return candidate_UPCs

def validate_upc(candidates):
    upcs = []
    dq_pattern = re.compile('[a-zA-Z\.\/]')
    for c in candidates:
        upcs += [re.sub('-', '', p[0]) for p in c if p[1] == 'CD' 
                 and len(re.sub('-', '', p[0])) in [8, 12, 13] 
                 and dq_pattern.search(p[0]) is None]
    return upcs

In [454]:
#Vectorize over a series of strings
def extract_probable_specifiers(text):
    if pd.isnull(text):
        text=''
    pattern = r"(([0-9A-Z])+[a-z]*([\\-]?[\\.*]?[0-9A-Z]*)*){2,}"
    matches = re.finditer(rg, text)
    unique_matches = set([match.group() for matchNum, match in enumerate(matches)])
    return list(unique_matches)

In [275]:
#Munge and infill whatever information we can extract from the recall descriptions

##UPCs loaded as strings; clean for whitespace & non-numeric chars
#provided_upcs = recalls['ProductUPCs_UPC'].str.replace(' |-|\.', '')
#provided_upcs[provided_upcs.isnull()] = ''
##extract UPCs from description; add where empty;
#inferred_upcs = [validate_upc(identify_probable_upc(d)) for d in recalls['Description']]
#merged_upcs = [inferred_upcs[i] + [provided_upcs[i]] if provided_upcs[i] else inferred_upcs[i] for i in range(len(inferred_upcs))]
#recalls['ProductUPCs_UPC'] = merged_upcs
recalls = recalls.rename(columns={'ProductUPCs_UPC': 'UPC'})
#Extract unit numbers from string phrases (e.g. "About 35")
num_units = recalls['Products_NumberOfUnits'].str.replace(',', '')
num_units = num_units.str.extract(r'(\d+)', expand=False).astype('float')
recalls['Products_NumberOfUnits'] = num_units
#Parse dates from strings
recalls['RecallDate'] = pd.to_datetime(recalls['RecallDate'])
recalls['LastPublishDate'] = pd.to_datetime(recalls['LastPublishDate'])
#Break the standardized titles into helpful fields; standardized
#titles take the form "[Company] recalls [product] due to [hazard]"
titles = recalls['Title'].str.split('Recall[s]|Due to', expand=True)
titles = titles.rename(columns={0: 'CompanyShortname', 1: 'ProductsShortname', 2: 'HazardAlt'})
recalls = pd.concat([recalls, titles], axis=1)

In [ ]:
#ItemNumber: {<NNP><:|\#>*<JJ>}
samples = recalls['Description'].sample(n=100)
candidates = [identify_probable_upc(s) for s in samples]
second_pass = [validate_upc(c) for c in candidates]

In [455]:
descs = [extract_probable_specifiers(dsc) for dsc in recalls['Description']]

In [456]:
descs

[['UPC',
  '11',
  'TH.023100-18',
  '032053118005',
  'TH.011800-18',
  '032053418006',
  '032053918001',
  'TH.091800-18',
  'TH.041800-18',
  'WaterSeal',
  '032053231001'],
 [],
 ['2019', '33.5', '92129', '2019.', '31', '15'],
 ['UPC',
  '656857123333',
  '656857123340',
  '653981740108',
  '653981740030',
  '656857123326',
  'BW204',
  '653981740092'],
 ['GB125-35'],
 ['UPC', '9211D', '4.5', '763795556991', 'XKT1747', 'SKU'],
 [],
 ['AXS', '2019', 'BMC', '8R17', 'FK6', '8R18', 'SLR01', '2018', 'DISC'],
 ['UPC', '2019', 'Light-Up', '18', '373', '19', '2018', '085761220034'],
 ['17', '36', '48'],
 ['099105125419', 'UPC', '099105125420', 'J8406', 'FA', '19'],
 ['13.6', '5.3'],
 ['STEAK',
  '2366917',
  '2366916',
  'SKU',
  'MS-5B-PG',
  'MR.',
  '2472264',
  '2472265',
  'SKUs',
  'MS-4B-PG'],
 ['24-Pack', '8-Pack'],
 ['60', '2019', '2018', 'XL.', '45', '2017', '8S'],
 ['2022',
  'STFS',
  'FireStop',
  '2016',
  'StoveTop',
  '13',
  '4516',
  '2016.',
  'DEC'],
 ['M0934Q',
  '38',

In [363]:
has_model = reports[~pd.isna(reports['Model Name or Number'])]
len(has_model)
#has_model.head(30)

35042

In [469]:
recalls['CompanyShortname'].iloc[0:10]

0            The Thompson’s Company 
1    Boston Warehouse Trading Corp. 
2                      Libbey Glass 
3        Thesaurus Global Marketing 
4            Bosch Thermotechnology 
5                          Hallmark 
6                         BCI Burke 
7                               BMC 
8                          Toysmith 
9               Hillsdale Furniture 
Name: CompanyShortname, dtype: object

In [260]:
reports = pd.read_csv('SPDB/IncidentReports.csv', encoding="ISO-8859-1", dtype='object')

In [476]:
bosch = reports[reports['Manufacturer / Importer / Private Labeler Name'].str.contains('Bosch') | reports['Brand'].str.contains('Bosch')]
bosch

,Report No.,Report Date,Sent to Manufacturer / Importer / Private Labeler,Publication Date,Category of Submitter,Product Description,Product Category,Product Sub Category,Product Type,Product Code,Manufacturer / Importer / Private Labeler Name,Brand,Model Name or Number,Serial Number,UPC,Date Manufactured,Manufacturer Date Code,Retailer,Retailer State,Purchase Date,Purchase Date Is Estimate,Incident Description,City,State,ZIP,Location,(Primary) Victim Severity,(Primary) Victim's Gender,My Relation To The (Primary) Victim,(Primary) Victim's Age (years),Submitter Has Product,Product Was Damaged Before Incident,Damage Description,Damage Repaired,Product Was Modified Before Incident,Have You Contacted The Manufacturer,If Not Do You Plan To,Answer Explanation,Company Comments,Associated Report Numbers
10,20191202-0EE2E-2147376236,12/2/2019,12/27/2019,1/10/2020,Consumer,Bosch Dishwasher Model # SHEM63W55N/10,Kitchen,Appliances,Dishwashers (214),214,BSH HOME APPLIANCES CORPORATION,Bosch,SHEM63W55N/10,FD980402890,NaN,NaN,NaN,Best Buy,North Carolina,4/28/2018,Yes,Bosch Dishwasher power cord caught on fire. Th...,Raleigh,North Carolina,27608,Home/Apartment/Condominium,"Incident, No Injury",Unknown,Unspecified,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,I contacted the manufacturer and the customer ...,(1/9/2020) BSH HOME APPLIANCES CORPORATION: Th...,NaN
113,20191113-6DFF6-2147376590,11/13/2019,12/13/2019,12/31/2019,Consumer,Bosch dishwasher with controls on the top. 3 ...,Kitchen,Appliances,Dishwashers (214),214,BSH HOME APPLIANCES CORPORATION,Bosch,SHP65TL5UC/09,FD961101714,NaN,NaN,FD961101714,Alber Lee Appliances,Washington,3/8/2017,Yes,Wife went to use the Bosh Dishwasher on the af...,Seattle,Washington,98136,Home/Apartment/Condominium,"Incident, No Injury",Female,My Spouse,NaN,Yes,No,NaN,NaN,No,Yes,NaN,Called about getting it serviced. Found out a...,(1/9/2020) BSH HOME APPLIANCES CORPORATION: Th...,NaN
131,20191111-4EB92-2147376627,11/11/2019,12/13/2019,12/31/2019,Consumer,Bosch 800 series electric wall oven,Kitchen,Appliances,Electric Ranges or Ovens (Excl Counter-top Ove...,278,BSH HOME APPLIANCES CORPORATION,Bosch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Danger when opening 2019 Bosch 800 series sing...,Brooklyn,New York,11217,Home/Apartment/Condominium,"Injury, No First Aid or Medical Attention Rece...",Male,Self,48,Yes,No,NaN,NaN,No,No,No,NaN,(1/9/2020) BSH HOME APPLIANCES CORPORATION: BS...,NaN
267,20191101-00021-2147376817,11/1/2019,11/19/2019,12/4/2019,Consumer,"Bosch Integra series dishwasher 18"" width, sta...",Kitchen,Appliances,Dishwashers (214),214,BSH HOME APPLIANCES CORPORATION,Bosch,SPX5ES55UC/07,FD9205,'00039,NaN,NaN,Appliances Connection,NaN,12/20/2012,Yes,Dishwasher Power cord caught fire,BOSTON,Massachusetts,02118,Home/Apartment/Condominium,"Incident, No Injury",Unspecified,Self,57,Yes,No,NaN,NaN,No,No,Yes,It was destroyed by firemen.,(12/5/2019) BSH HOME APPLIANCES CORPORATION: T...,NaN
328,20191015-16DFD-2147377150,10/15/2019,11/12/2019,11/26/2019,Consumer,Bosch Dishwasher Model #SHX68T55UC/02\r\n\r\nO...,Kitchen,Appliances,Dishwashers (214),214,BSH HOME APPLIANCES CORPORATION,Bosch,SHX68T55UC/02,FD9601,NaN,NaN,03626,PC Richard and Son,New Jersey,1/17/2016,NaN,Bosch Dishwasher Model #SHX68T55UC/02\r\n\r\nO...,Stewartsville,New Jersey,08886,Home/Apartment/Condominium,"Incident, No Injury",Unspecified,Self,NaN,Yes,No,NaN,NaN,No,Yes,NaN,The manufacturer is saying our serial number i...,(12/5/2019) BSH HOME APPLIANCES CORPORATION: A...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40817,20110415-9688E-2147480171,4/15/2011,NaN,5/6/2011,Consumer,"Bosch stainless steel dishwasher, model # SHY5...",Kitchen,Appliances,Dishwashers (214),214,BSH HOME APPLIANCES CORPORATION,Bosch,SHY56A050C/14,FD850100955,'N/A,NaN,NaN,NaN,NaN,NaN,NaN,"Bosch dishwasher, model # SHY56A050uc/4, Seria...",Albuquerque,New Mexico,87111,Home

In [483]:
bosch_recalls = recalls[recalls['CompanyShortname'].str.contains('Bosch') | recalls['Manufacturers_Name'].str.contains('Bosch') 
   | recalls['Importers_Name'].str.contains('Bosch') | recalls['Distributors_Name'].str.contains('Bosch') | 
   recalls['Retailers_Name'].str.contains('Bosch')]

In [507]:
bosch_recalls.head()
bosch_recalls.dtypes
bosch_recalls.loc[3358, :]['Description']

"Bosch and Siemens Model Dishwashers Recalled by BSH Home Appliances Corporation Due to Fire Hazard NEWS from CPSC U.S. Consumer Product Safety Commission Office of Information and Public Affairs Washington, DC 20207 FOR IMMEDIATE RELEASE January 15, 2009 Release #09-100 Firm's Recall Hotline: (800) 856-9226 Firm’s Media Hotline: (310) 309-6611 CPSC Recall Hotline: (800) 638-2772 CPSC Media Contact: (301) 504-7908 Bosch® and Siemens® Model Dishwashers Recalled by BSH Home Appliances Corporation Due to Fire Hazard WASHINGTON, D.C. - The U.S. Consumer Product Safety Commission, in cooperation with the firm named below, today announced a voluntary recall of the following consumer product. Consumers should stop using recalled products immediately unless otherwise instructed. It is illegal to resell or attempt to resell a recalled consumer product. Name of Product: Bosch® and Siemens® Model Dishwashers Units: About 476,500 Manufacturer: BSH Home Appliances Corp., of Huntington Beach, Calif.

In [117]:
reports.columns

Index(['Report No.', 'Report Date',
       'Sent to Manufacturer / Importer / Private Labeler', 'Publication Date',
       'Category of Submitter', 'Product Description', 'Product Category',
       'Product Sub Category', 'Product Type', 'Product Code',
       'Manufacturer / Importer / Private Labeler Name', 'Brand',
       'Model Name or Number', 'Serial Number', 'UPC', 'Date Manufactured',
       'Manufacturer Date Code', 'Retailer', 'Retailer State', 'Purchase Date',
       'Purchase Date Is Estimate', 'Incident Description', 'City', 'State',
       'ZIP', 'Location', '(Primary) Victim Severity',
       '(Primary) Victim's Gender', 'My Relation To The (Primary) Victim',
       '(Primary) Victim's Age (years)', 'Submitter Has Product',
       'Product Was Damaged Before Incident', 'Damage Description',
       'Damage Repaired', 'Product Was Modified Before Incident',
       'Have You Contacted The Manufacturer', 'If Not Do You Plan To',
       'Answer Explanation', 'Company Comments

In [261]:
len(reports)

41542

In [265]:
len(recalls[pd.to_datetime(recalls['RecallDate']) > min(pd.to_datetime(reports['Report Date']))])

2535

In [270]:
min(pd.to_datetime(reports['Report Date']))

Timestamp('2011-03-11 00:00:00')

In [271]:
reports.tail()

,Report No.,Report Date,Sent to Manufacturer / Importer / Private Labeler,Publication Date,Category of Submitter,Product Description,Product Category,Product Sub Category,Product Type,Product Code,Manufacturer / Importer / Private Labeler Name,Brand,Model Name or Number,Serial Number,UPC,Date Manufactured,Manufacturer Date Code,Retailer,Retailer State,Purchase Date,Purchase Date Is Estimate,Incident Description,City,State,ZIP,Location,(Primary) Victim Severity,(Primary) Victim's Gender,My Relation To The (Primary) Victim,(Primary) Victim's Age (years),Submitter Has Product,Product Was Damaged Before Incident,Damage Description,Damage Repaired,Product Was Modified Before Incident,Have You Contacted The Manufacturer,If Not Do You Plan To,Answer Explanation,Company Comments,Associated Report Numbers
41537,20110311-DBB63-2147481650,3/11/2011,NaN,4/1/2011,Consumer,"Pampers Swaddlers New Baby with Dry Max, Size 1-2",Baby,Nursery Equipment & Supplies,Diapers (1512),1512,The Procter & Gamble Company,Pampers,"Swaddlers New Baby with Dry Max, Size 1-2",0363U017630008,NaN,NaN,NaN,Sam's Club,Indiana,3/1/2011,NaN,"Since he was born two months ago, we have been...",Lebanon,Indiana,46052,Home/Apartment/Condominium,"Injury, First Aid Received by Non-Medical Prof...",Male,My Child,0,No,No,NaN,NaN,No,Yes,NaN,I will be writing a letter to the company foll...,(4/1/2011) The Procter & Gamble Company: Thank...,NaN
41538,20110311-E518D-1170349,3/11/2011,NaN,4/1/2011,Consumer,Airzone Pop stick. Bright/ lime green with bla...,Toys & Children,Toys,Pogo Sticks (1310),1310,BRAVO SPORTS,Airzone,Pop stick,unknown,NaN,9/24/2010,054CH,Walmart Stores Inc.,Arkansas,2/24/2011,Yes,My nine year old daughter was using her Airzon...,Santa Maria,California,93455,Home/Apartment/Condominium,"Injury, First Aid Received by Non-Medical Prof...",Female,My Child,9,Yes,No,NaN,No,No,No,No,NaN,(3/31/2011) BRAVO SPORTS: Thank you for transm...,NaN
41539,20110313-2E80A-2147481520,3/13/2011,NaN,4/1/2011,Consumer,Beige colored dog toy which has a bone shaped ...,Toys & Children,Toys,"Dolls, Plush Toys, and Action Figures (1394)",1394,Target,Target,087011014,F16805009,NaN,5/1/2009,05/09,Gift from Target,California,NaN,NaN,"The cord which attaches the ""remote"" to the wa...",Moraga,California,94556,Home/Apartment/Condominium,"Incident, No Injury",Female,My Child,4,Yes,No,NaN,NaN,No,No,No,NaN,NaN,NaN
41540,20110313-26730-2147481529,3/13/2011,NaN,4/1/2011,Consumer,Toro CCR 3650 Snowblower,Yard & Garden,Gardening & Landscaping,"Snow Throwers, Blowers (1406)",1406,The Toro Company,Toro,38518,220008051,NaN,NaN,Not Known,Home Depot,Ohio,2/1/2003,Yes,"A few weeks ago, we started experiencing a ver...",NaN,United States,NaN,Home/Apartment/Condominium,"No Incident, No Injury",Unspecified,NaN,NaN,Yes,No,NaN,NaN,No,No,Yes,I plan on contacting a Toro authorized service...,NaN,NaN
41541,20110311-6B7FC-2147481644,3/11/2011,NaN,4/1/2011,Consumer,Harman Oakwood Freestanding wood stove. Manufa...,Home Maintenance and Structures,"Heating, Ventilation & Air Conditioning",Coal or Wood-burning Stoves (367),367,HARMAN STOVE COMPANY,Harman,Oakwood,NaN,NaN,2/1/2010,NaN,Wrights Chimney and Stove,North Carolina,6/1/2010,Yes,I own a Harman Oakwood freestanding wood burni...,NaN,United States,NaN,Home/Apartment/Condominium,"No Incident, No Injury",Unspecified,NaN,NaN,Yes,No,NaN,NaN,No,No,No,The stove weighs 500 pounds. It isn't going an...,NaN,NaN


In [467]:
regex = r"(?<![\"=\w])(?:[^\W_]+)(?![\"=\w])"
#rg = r"([0-9A-Z]{2,}[a-z]*[\\-]?[\\.*]?[0-9A-Z]*)+"
#rg = r"([A-Za-z]?[\s]?[#]?[:]?[\s]?[0-9A-Z]{2,}[\\-]?[\\.*]?)+"
rg = r"(([0-9A-Z])+[a-z]*([\\-]?[\\.*]?[0-9A-Z]*)*){2,}"
test_str = "WA50F9A65DSW/A2"
test = recalls['Description'].iloc[0]
print(test)

matches = re.finditer(rg, test_str)

for matchNum, match in enumerate(matches):
    print (match.group())

This recall involves Thompson’s WaterSeal waterproofing wood and masonry protectors in aerosol cans. The products are used to coat exterior wood to prevent water damage. The aerosol cans are 11 ounces and have a green or blue cap. “Thompson’s WaterSeal,” “Wood Protector” or “Masonry Protector,” the item number and UPC code are printed on the can. Thompson’s WaterSeal Waterproofing Wood Protector Aerosol Item #: TH.041800-18 UPC: 032053418006 Thompson’s WaterSeal Signature Series Aerosol Item #: TH.091800-18 UPC: 032053918001 Thompson’s WaterSeal Waterproofer Plus Wood Protector Aerosol Item #: TH.011800-18 UPC: 032053118005 Thompson’s WaterSeal Waterproofer Plus Masonry Protector Aerosol Item #: TH.023100-18 UPC: 032053231001
WA50F9A65DSW
A2


In [463]:
small_reports = reports.sample(500)

In [458]:
small_reports

,Report No.,Report Date,Sent to Manufacturer / Importer / Private Labeler,Publication Date,Category of Submitter,Product Description,Product Category,Product Sub Category,Product Type,Product Code,Manufacturer / Importer / Private Labeler Name,Brand,Model Name or Number,Serial Number,UPC,Date Manufactured,Manufacturer Date Code,Retailer,Retailer State,Purchase Date,Purchase Date Is Estimate,Incident Description,City,State,ZIP,Location,(Primary) Victim Severity,(Primary) Victim's Gender,My Relation To The (Primary) Victim,(Primary) Victim's Age (years),Submitter Has Product,Product Was Damaged Before Incident,Damage Description,Damage Repaired,Product Was Modified Before Incident,Have You Contacted The Manufacturer,If Not Do You Plan To,Answer Explanation,Company Comments,Associated Report Numbers
30044,20121023-A72D3-2147462598,10/24/2012,10/31/2012,11/15/2012,Consumer,Ultra Wash Dishwasher model 665.1435.3301,Kitchen,Appliances,Dishwashers (214),214,Sears Holdings Management Corporation,Kenmore,665.1435.3301/ Ultra Wash Dishwasher,FR1701330,NaN,NaN,NaN,New home,NaN,NaN,NaN,Control panel on my Kenmore 665.1435.3301 star...,Midlothian,Texas,76065,Home/Apartment/Condominium,"Incident, No Injury",Female,My Spouse,59,Yes,No,NaN,NaN,No,No,Yes,Will contact them for replacement of dishwashe...,(10/31/2012) Sears Holdings Management Corpora...,NaN
28680,20130106-105BB-2147460176,1/6/2013,1/18/2013,2/11/2013,Consumer,Sesame Street Chilly Teething Ring,Baby,Nursery Equipment & Supplies,Pacifiers or Teething Rings (1525),1525,Munchkin Inc.,Sesame Street,Sesame Street Chilly Teether,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Munchkin Sesame Street Chilly Teether ring...,Greensboro,North Carolina,NaN,Home/Apartment/Condominium,"Injury, No First Aid or Medical Attention Rece...",Male,My Child,1,Yes,No,NaN,NaN,No,No,Yes,Saw a previous complaint about this product. ...,(1/22/2013) Munchkin Inc.: Thank you for bring...,NaN
40077,20110522-71454-2147478939,5/22/2011,NaN,6/14/2011,Consumer,foam mattress,"Furniture, Furnishings & Decorations","Bedroom, Bedding & Bath",Nonbaby Mattresses (4009),4009,CLASSIC SLEEP PRODUCTS INC.,BOB OPedic,pt1027xxd,b675b03f031,NaN,11/16/2010,11/16/2010,bob opedic,New Jersey,5/15/2011,NaN,I bought a foam mattress from Bobs discount fu...,Bayside,New Jersey,11361,Home/Apartment/Condominium,"Injury, Level of care not known",Male,Unspecified,NaN,Yes,No,NaN,NaN,No,No,Yes,will advise of situation on Monday,NaN,NaN
31621,20120730-D8ADD-2147465224,7/30/2012,8/10/2012,8/24/2012,Consumer,rocker bottom shoes,Clothing and Accessories,Shoes and other Footwear,Footwear (1615),1615,"Skechers USA, Inc.",Shape-ups,Shape-Ups XF-Accelerators,12320,NaN,7/1/2010,NaN,Sketcher's Website,NaN,2/25/2012,NaN,"On April 30,2012 I was walking to my car and f...",Clinton,South Carolina,29325,Mobile/Manufactured Home,"Injury, Seen by Medical Professional",Female,Self,33,Yes,No,NaN,NaN,No,No,No,I kept the shoes to have just incase I need th...,"(8/10/2012) Skechers USA, Inc.: We appreciate ...",NaN
9362,20170225-5B19D-2147405738,2/25/2017,3/9/2017,3/30/2017,Consumer,"Calphalon 6""chef knife individually sold \r\n\...",Kitchen,Knives & Slicers,"Knives, Not Elsewhere Classified (464)",464,Newell Rubbermaid Inc.,Calphalon,"6""chef",0863115,NaN,NaN,NaN,Gift,NaN,12/1/2016,Yes,Knife broke when my husband was using it. A c...,Lauderhill,Florida,33319,Home/Apartment/Condominium,"Incident, No Injury",Male,My Spouse,46,Yes,No,NaN,NaN,No,No,Yes,I didn't realize other models were recalled du...,(4/18/2017) Newell Rubbermaid Inc.: We are in ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29595,20121120-AEF0D-2147461748,11/20/2012,12/12/2012,12/28/2012,Consumer,"Target- Classic Wall Shelf 36"" in white","Furniture, Furnishings & Decorations",Storage & Organization,"Cabinets, Racks, Room Dividers and Shelves, Ne...",4056,Target,Target,"Target- Clas

In [462]:
small_reports.to_csv('report_sample.csv')